# 다중분류 (뉴스 기사 분류 문제)
이전 섹션에서 본 예제는 벡터 입력을 받아서 두개의 클래스로 분류하는 문제였다.  
두개 이상의 클래스(종류,분류)로 나위어 질 경우 다중분류라는 방식을 사용한다.  
로이터 뉴스를 46개의 토픽으로 분류하는 예제를 통해 다중분류에 대해 확인해보자.  

## 로이터 데이터 셋
 - 1986년에 로이터에서 공개한 짧은 뉴스 기사와 토픽의 집합인 로이터 데이터셋을 사용.
 - 텍스트 분류를 위해 널리 사용되는 간단한 데이터셋이다.
 - 46개의 토픽이 있으며 어떤 토픽은 다른 것에 비해 데이터가 많다. 각 토픽은 훈련 세트에 최소한 10개의 샘플을 가지고 있다.